# Notebook 6: Quantitative Evaluation

This notebook computes evaluation metrics (PSNR, SSIM, LPIPS, Edge Accuracy) for all restoration methods and stores results in a CSV file.


In [ ]:
import sys
import os
from pathlib import Path
import numpy as np
import cv2
import pandas as pd
from tqdm import tqdm
import lpips

sys.path.append('..')
from utils import (
    load_image, get_image_files, 
    calculate_psnr, calculate_ssim, 
    calculate_lpips, calculate_edge_accuracy
)

print("Libraries imported successfully!")


## 1. Setup Paths and Initialize LPIPS Model


In [ ]:
# Paths
GROUND_TRUTH_DIR = Path('../data/ground_truth')
METHODS = {
    'PDE': Path('../methods/PDE/results'),
    'Patch': Path('../methods/Patch/results'),
    'Deep': Path('../methods/Deep/results'),
    'Hybrid': Path('../methods/Hybrid/results')
}

# Initialize LPIPS model (only once)
print("Initializing LPIPS model...")
lpips_model = lpips.LPIPS(net='alex')
print("LPIPS model ready!")

# Get all ground truth files
gt_files = get_image_files(GROUND_TRUTH_DIR)
print(f"Found {len(gt_files)} ground truth images")


## 2. Compute Metrics for All Methods


In [ ]:
# Initialize results list
results = []

print("Computing metrics for all methods...")

for gt_path in tqdm(gt_files, desc="Evaluating"):
    gt_img = load_image(gt_path)
    base_name = gt_path.stem
    
    result_row = {'image_name': base_name}
    
    # Evaluate each method
    for method_name, method_dir in METHODS.items():
        method_result_path = method_dir / f"{base_name}.png"
        
        if method_result_path.exists():
            method_result = load_image(method_result_path)
            
            # Compute metrics
            psnr_val = calculate_psnr(gt_img, method_result)
            ssim_val = calculate_ssim(gt_img, method_result)
            lpips_val = calculate_lpips(gt_img, method_result, lpips_model)
            edge_acc = calculate_edge_accuracy(gt_img, method_result)
            
            result_row[f'{method_name}_PSNR'] = psnr_val
            result_row[f'{method_name}_SSIM'] = ssim_val
            result_row[f'{method_name}_LPIPS'] = lpips_val
            result_row[f'{method_name}_EdgeAcc'] = edge_acc
        else:
            # If result doesn't exist, set to NaN
            result_row[f'{method_name}_PSNR'] = np.nan
            result_row[f'{method_name}_SSIM'] = np.nan
            result_row[f'{method_name}_LPIPS'] = np.nan
            result_row[f'{method_name}_EdgeAcc'] = np.nan
    
    results.append(result_row)

# Create DataFrame
df = pd.DataFrame(results)
print(f"\nComputed metrics for {len(df)} images")


## 3. Summary Statistics


In [ ]:
# Compute summary statistics for each method
summary_stats = []

for method_name in METHODS.keys():
    method_stats = {
        'Method': method_name,
        'PSNR_mean': df[f'{method_name}_PSNR'].mean(),
        'PSNR_std': df[f'{method_name}_PSNR'].std(),
        'SSIM_mean': df[f'{method_name}_SSIM'].mean(),
        'SSIM_std': df[f'{method_name}_SSIM'].std(),
        'LPIPS_mean': df[f'{method_name}_LPIPS'].mean(),
        'LPIPS_std': df[f'{method_name}_LPIPS'].std(),
        'EdgeAcc_mean': df[f'{method_name}_EdgeAcc'].mean(),
        'EdgeAcc_std': df[f'{method_name}_EdgeAcc'].std()
    }
    summary_stats.append(method_stats)

summary_df = pd.DataFrame(summary_stats)

print("\n=== Summary Statistics ===")
print(summary_df.to_string(index=False))

# Display best method for each metric
print("\n=== Best Methods ===")
print(f"Best PSNR: {summary_df.loc[summary_df['PSNR_mean'].idxmax(), 'Method']} ({summary_df['PSNR_mean'].max():.4f})")
print(f"Best SSIM: {summary_df.loc[summary_df['SSIM_mean'].idxmax(), 'Method']} ({summary_df['SSIM_mean'].max():.4f})")
print(f"Best LPIPS (lowest): {summary_df.loc[summary_df['LPIPS_mean'].idxmin(), 'Method']} ({summary_df['LPIPS_mean'].min():.4f})")
print(f"Best Edge Accuracy: {summary_df.loc[summary_df['EdgeAcc_mean'].idxmax(), 'Method']} ({summary_df['EdgeAcc_mean'].max():.4f})")


## 4. Save Results to CSV


In [ ]:
# Save detailed results
results_csv_path = Path('../evaluation_results.csv')
df.to_csv(results_csv_path, index=False)
print(f"Detailed results saved to: {results_csv_path}")

# Save summary statistics
summary_csv_path = Path('../evaluation_summary.csv')
summary_df.to_csv(summary_csv_path, index=False)
print(f"Summary statistics saved to: {summary_csv_path}")

print("\n✓ Evaluation complete!")
